In [ ]:
import os
import sys
import re

sys.path.append(os.path.join('..'))
sys.path.append(os.path.join('..', 'src'))
sys.path.append(os.path.join('..', 'src', 'libs'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from matplotlib.colors import ListedColormap, LogNorm
from copy import deepcopy
import seaborn as sns

import hera
from libs import parser as parse
from libs import titration as titi
from libs import util as utl

from ami_utils import parse as pa

import matplotlib as mpl
mpl.rcParams['font.family'] = 'Arial'

### Generate Titration from config

In [ ]:
# exp_names = ['LUB045', 'LUB035', 'LUB060'] # Change counter ion Eu
# exp_names = ['LUB045', 'LUB157', 'LUB100', 'LUB099'] # Eu ratio to Mebip (hcl decomplex)
# exp_names = ['LUB045', 'LUB100', 'LUB157'] # Eu ratio to Mebip (complex)
# exp_names = ['LUB055', 'LUB067'] # Copper II
# exp_names = ['LUB005', 'LUB100', 'LUB099', 'LUB048', 'LUB055'] # Different metals (Zn, Eu[OTf + ClO4], Cu) and endpoint
# exp_names = ['LUB005', 'LUB100', 'LUB099', 'LUB055'] # Different metals (Zn, Eu, Cu) [all OTf] and endpoint
# exp_names = ['LUB005', 'LUB045', 'LUB067'] # Different metals (Zn, Eu, Cu) [all OTf] and order_complex=2
# exp_names = ['LUB005', 'LUB021', 'LUB038', 'LUB192'] # Acids in Zn + TEAC
# exp_names = ['LUB005', 'LUB021', 'LUB038', 'LUB069', 'LUB035', 'LUB048', 'LUB055'] # Acids in Zn + TEAC + Different metals
# exp_names = ['LUB005', 'LUB045', 'LUB055', 'LUB067', 'LUB157']
# exp_names = ['LUB005', 'LUB038', 'LUB069'] # Acids in Zn
# exp_names = ['LUB055', 'LUB101'] # HCl-complex Cu(OTf3)
# exp_names = ['LUB005', 'LUB102'] # HCl-complex Zn(OTf3)2
# exp_names = ['LUB005', 'LUB055', 'LUB101', 'LUB102']
# exp_names = ['LUB005', 'LUB099', 'LUB045']
# exp_names = ['LUB096', 'LUB141']
# exp_names = ['LUB005', 'LUB193', 'LUB195', 'LUB150'] # Comparison Mebip, MBP Zn and MBP Fe2+ Hcl decomplex
# exp_names = ['LUB190', 'LUB194', 'LUB150'] # Comparison MBP Zn, MBP Fe2+ and Cu2+ complex
# exp_names = ['LUB045', 'LUB005', 'LUB157', 'LUB067', 'LUB055'] # Mebip decomplex hcl
# exp_names = ['LUB045', 'LUB005', 'LUB055'] # Mebip complex metals
exp_names = ['LUB261']
# exp_names = ['LUB195', 'LUB193']

In [ ]:
hera.initialize_titrations(exp_names)

In [ ]:
dict_titrants_exps = hera.dict_titrants_exps.copy()
dict_titrations_exps = deepcopy(hera.dict_titrations_exps)

In [ ]:
dict_titrants_exps

In [ ]:
for k, v in dict_titrations_exps.items():
    print(f'\n\n{k}:\n')
    for t in v[0]:
        print(t.df_params)

### Import data associated

In [ ]:
#TODO: Check parser get the first line of text file.

print(exp_names)
exps_reps = parse.get_exp_reps(exp_names)
data = pa.get_data(pa.path_constructor('uv', *exps_reps), 'uv')
data

### Distinguish from different titrations and merge titration params with data

In [ ]:
new_data = utl.merge_data_titration_params(data, dict_titrations_exps)
new_data

In [ ]:
new_data.titrant.unique()

In [ ]:
# # Reorganise data to ensure metal and acid titrations can be distinguished

# def characterise_metal_acid_titrations(data):
#     data['volume_change'] = data.titrant.apply(lambda x: float(re.match('[0-9.]+', x).group()))
#     # data.drop(columns=['titrant'], inplace=True)
#     data.titrant  = data.titrant.apply(lambda x: re.search('[a-z]+', x).group() if re.search('[a-z]+', x) else 'm')
    
#     return data

# def merge_data_titration_params(data):
    
#     data = characterise_metal_acid_titrations(data)
    
#     list_df = []
#     for exp in data.name.unique():

#         # Merge titration with data of each experiment and populate concat list
#         for k, v in dict_titrations_exps.items():
#             if k.casefold() in exp.casefold():
#                 data_exp = data.loc[data.name == exp]
#                 for titrations in v[0]:
#                     df = titrations.df_params
#                     list_df.append(
#                          df.loc[:, ['volume_change', 'g_h', 'guest_name', 'host_name', 'rank']] \
#                            .merge(data_exp, how='inner', on=["volume_change"]))

#     new_data = pd.concat(list_df, axis=0).fillna(method='bfill')
    
#     return new_data

# new_data = merge_data_titration_params(data)
# new_data

In [ ]:
# Reorganise data to ensure metal and acid titrations can be distinguished

# def characterise_metal_acid_titrations(data):
#     data['volume_change'] = data.titrant.apply(lambda x: float(re.match('[0-9.]+', x).group()))
#     data.titrant  = data.titrant.apply(lambda x: re.search('[a-z]+', x).group() if re.search('[a-z]+', x) else 'm')
    
#     return data

# def merge_data_titration_params(data):
    
#     data = characterise_metal_acid_titrations(data)
#     print(data)
    
#     list_df = []
#     for exp in data.name.unique():

#         for k, v in dict_titrations_exps.items():
#             if k.casefold() in exp.casefold():

#                 data_exp = data.loc[data.name == exp]
#                 print(data_exp)
#                 guest_not_metal = [tit for tit in data_exp.titrant.unique() if tit != 'm'] # Here you could use sets - 1 line and faster
#                 guest_not_metal = guest_not_metal[0] if guest_not_metal else 'None'
#                 for titrations in v[0]:
#                     df = titrations.df_params
#                     if guest_not_metal in titrations.guest.name.casefold():
#                         # df = titrations.df_params
#                         print()
#                         list_df.append(
#                              df.loc[:, ['volume_change', 'g_h']] \
#                                .merge(data_exp[data_exp.titrant==guest_not_metal], how='outer', on=["volume_change"]))
#                     else:
#                         # df = titrations.df_params
#                         list_df.append(
#                             df.loc[:, ['volume_change', 'g_h']] \
#                               .merge(data_exp[data_exp.titrant!=guest_not_metal], how='outer', on=["volume_change"]))

#     new_data = pd.concat(list_df, axis=0).fillna(method='bfill')
    
#     return new_data

# new_data = merge_data_titration_params(data)
# new_data

### Generate peak tracking data

In [ ]:
# 350 nm for Eu
# 335 nm for Zn
# 345 nm for Cu
# 342 nm for Mg bidentate
# 345 nm for Cu bidentate
# 350 nm for Zn bidentate
# 350/495 nm for Fe2+ bidentate
# 555 nm for RhodB
# wl_0=313
dict_df = utl.tracking_df(new_data, [350], exps_reps, wl_0=313, complex_tit=True)
dict_df

In [ ]:
df = utl.combine_track_data(new_data, dict_df)
df

In [ ]:
if True:
    # Write this into the back - perhaps
    def remove_unrelated_wl_track(df, labels_mlct):
        for name, mlct in labels_MLCT.items():
            df.drop(df.loc[(df.name.str.contains(name)) & ~(df.wl_track == str(mlct))].index, inplace=True)
        df.reset_index(drop=True)
        return df
    
    # Name and associated mlct band wl for experiment
    # labels_MLCT = dict(zip(['LUB005', 'LUB045', 'LUB055', 'LUB067', 'LUB157'], [335, 350, 345, 345, 350]))
    # labels_MLCT = dict(zip(['LUB005', 'LUB193', 'LUB195'], [335, 350, 350])) # Mebip vs bidentate hcl
    # labels_MLCT = dict(zip(['LUB190', 'LUB194', 'LUB150'], [350, 350, 345])) # Mebip vs bidentate complex
    labels_MLCT = dict(zip(['LUB005', 'LUB045', 'LUB055', 'LUB067', 'LUB157', 'LUB193', 'LUB195'], 
                           [335, 350, 345, 345, 350, 350, 350])) # All Mebip and MBP decomplex
    df = remove_unrelated_wl_track(df, labels_MLCT)

### Visualise Data

#### ZnMebip vs Acids relationship

#### Metal vs HCl relationship

##### Line Graphs

In [ ]:
# Legend used for graph - super manuel

# legend_labels = ['Eu(EH-Mebip)$_{2}$', 'Eu(EH-Mebip)$_{2}$ - Excees Eu$^{3+}$', 'Eu(EH-Mebip)$_{3}$',]
# legend_labels = ['Zn(EH-Mebip)$_{2}$', 'Eu(EH-Mebip)$_{2}$', 'Cu(EH-Mebip)$_{2}$']
# legend_labels = ['Zn(EH-Mebip)$_{2}$', 'Cu(EH-Mebip)', 'Eu(EH-Mebip)$_{2}$ - Excees Eu$^{3+}$']
# legend_labels = ['HCl', 'TEAC', 'TFSA', 'TFA']
# legend_labels = ['Zn(EH-Mebip)$_{2}$', 'Eu(EH-Mebip)$_{2}$', 'Cu(EH-Mebip)$_{2}$', 'Cu(EH-Mebip)', 'Eu(EH-Mebip)$_{3}$']
# legend_labels = ['Eu(EH-Mebip)$_{2}$', 'Zn(EH-Mebip)$_{2}$', 'Eu(EH-Mebip)$_{3}$', 'Cu(EH-Mebip)', 
                 # 'Cu(EH-Mebip)$_{2}$']
# legend_labels = ['Eu(EH-Mebip)$_{2}$']
# legend_labels = ['Zn(EH-Mebip)$_{2}$']
# legend_labels = ['Cu(EH-Mebip)$_{2}$']
# legend_labels = ['Zn(OTf)$_{2}$']
# legend_labels = ['Eu(OTf)$_{3}$', 'Zn(OTf)$_{2}$', 'Cu(OTf)$_{2}$']
# legend_labels = ['Fe(EH-MBP)$_{3}$', 'Zn(EH-MBP)$_{3}$']
# legend_labels = ['Zn(OTf)$_{2}$', 'Fe(OTf)$_{3}$', 'Cu(OTf)$_{2}$']
legend_labels = ['Fe(II)(OTf)$_{2}$', 'Fe(III)(OTf)$_{3}$']
# legend_labels = ['350 nm', '495 nm']
# legend_labels = ['HCl']
# legend_labels = ['100 min', '5 min']

# Designate grouper (to group titrations according to host or guest)
grouper = 'host_name'

In [ ]:
print(sns.plotting_context("paper"))
sns.plotting_context("talk")

In [ ]:
# Plot in seperate figures
%matplotlib qt
# BG_WHITE = "#fafaf5"
with sns.plotting_context("paper", font_scale=1.75, 
                          rc={
                              "lines.linewidth": 2, 'lines.markersize': 8, 'patch.linewidth': 1.825
                          }):
    
    # Set colors for graph
    PalSet = sns.color_palette('deep', len(df.name.unique()))
    m_colors = PalSet.as_hex()
    m_colors = ['#ADE8F4', '#48CAE4', '#0096C7', '#0077B6', '#023E8A', '#96b1ad', '#439088'] # Mebip and MBP
    # m_colors = ['#96b1ad', '#439088', '#005852'] # MBP only
    
    # Plot data of interest
    to_plot=df[['g_h', 'host_name', 'guest_name', 'name', 'value', 'rank', 'wl_track']].drop_duplicates()
    to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]
    to_plot.sort_values(by='name', key=lambda column: column.map(lambda e: exp_names.index(e)), inplace=True) # Sort dataframe according to index provided

    for (exp, group) in to_plot.groupby([grouper], as_index=False, sort=False):
        print(exp)
        print(group.name.unique())
        print(group)
        
        fig, ax = plt.subplots() # Set a size once deteremined !
        # fig.patch.set_facecolor(BG_WHITE)
        # ax.set_facecolor(BG_WHITE)

        ax.set_xlabel('[Metal]:[EH-Mebip]')
        # ax.set_xlabel('[Metal]:[EH-MBP]')
        # ax.set_xlabel('[Zn(OTf)$_{2}$]:[EH-Mebip]')
        # ax.set_xlabel('[Zn(OTf)$_{2}$]:[PBA-$\it{co}$-MBA$_{4.5}$]')
        # ax.set_xlabel('[Eu(OTf)$_{3}$]:[EH-Mebip]') 
        # ax.set_xlabel('[Cu(OTf)$_{2}$]:[EH-Mebip]') 
        # ax.set_xlabel('[HCl]:[Fe(EH-MBP)$_{3}$]')
        # ax.set_xlabel('[HCl]:[Zn(EH-MBP)$_{3}$]')
        # ax.set_xlabel('[Fe(OTf)$_{2}$]:[EH-MBP]')
        # ax.set_xlabel('[Zn(OTf)$_{2}$]:[EH-MBP]')
        # ax.set_xlabel('[Zn(OTf)$_{2}$]:[Zn(EH-Mebip)$_{2}$]')
        # ax.set_xlabel('[HCl]:[Zn(EH-Mebip)$_{2}$]')
        # ax.set_xlabel('[HCl]:[Eu(EH-Mebip)$_{x=2,3}$]')
        # ax.set_xlabel('[HCl]:[Complex]')
        # ax.set_xlabel('[HCl]:[Fe(EH-MBP)$_{3}$]')
        # ax.set_xlabel('[hnhb]:[Eu(EH-Mebip)$_{2}$]')
        # ax.set_xlabel('[dipea]:[mbtt]')
        ax.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontsize=20)
        # ax.set_ylabel(r'$\Delta A_{555nm}$')
        # ax.set_ylabel(r'$\Delta A_{\pi\rightarrow\pi^{*}}$')
        # ax.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$')
        ax.spines[['right', 'top']].set_visible(False)
        # ax.spines[['left', 'right', 'top']].set_visible(False)
        # ax.spines[['bottom']].set_linewidth(2)
        # ax.tick_params(length=4, axis='both')
        
        # Tick paramters
        # ax.xaxis.set_major_locator(MultipleLocator(0.5))
        # Metal regime
        # ax.xaxis.set_major_locator(MultipleLocator(0.1))
        ax.xaxis.set_minor_locator(AutoMinorLocator(2))
        ax.yaxis.set_minor_locator(AutoMinorLocator(2))
       
        # acid regime
        # ax.xaxis.set_major_locator(MultipleLocator(1))
        # ax.xaxis.set_minor_locator(AutoMinorLocator(2))
        # ax.yaxis.set_minor_locator(AutoMinorLocator(2))
        # ax.tick_params(length=4, axis='both')
        # ax.tick_params(length=2, axis='both', which='minor')
        
        # ax.grid(axis='y', linestyle=':', lw=1, color='grey')
        # ax.tick_params(axis='y',length=0)
        ax.set_axisbelow(True)

        # Verticle lines indicating orders of complex for metal titrations
        # for el in [0.333, 0.5]:
        #     ax.axvline(x=el, ls='--', lw=1.5, color='black', alpha=0.4)
        # if group.host_name.unique()[0] == 'EH-Mebip':
        #     for el in [0.333, 0.5, 1]:
        #         ax.axvline(x=el, ls='--', lw=1.5, color='black', alpha=0.4)
        
        # lnplt = sns.lineplot(group, x="g_h", y="value", hue='name', ax=ax, palette='deep', legend=True)
        lnplt = sns.lineplot(group, x="g_h", y="value", hue='name', hue_order=exp_names, ax=ax, palette=m_colors, legend=True)
        
        # Legend - manual if legend required :(
        # new_title = 'Irradiation Time'
        bbox_to_anchor=(0.55, 0.45, 0.5, 0.5)
        # ax.legend(edgecolor='w', framealpha=0, loc='best')
        ax.legend(edgecolor='w', framealpha=0, bbox_to_anchor=bbox_to_anchor)
        # sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
        # lnplt.legend_.set_title(new_title)
        # Replace labels
        for text, label in zip(lnplt.legend_.texts, legend_labels):
            text.set_text(label)
        
        # Scatter plot for hollow spheres - matplotlib
        for (exp, group), col in zip(group.groupby(['name', 'guest_name', 'rank'], as_index=False, sort=False), m_colors):
            group_mean = group.groupby('g_h', as_index=False).agg({'value':'mean'})
            ax.scatter(group_mean.g_h, group_mean.value, edgecolors=col, facecolors="none", alpha=0.7, label=exp)

        fig.tight_layout()
        print(fig.get_size_inches())

In [ ]:
# %matplotlib qt
# # TODO: Move to plotting individually for simplicity (all complex on one graph and all decomplex on another seperate figs)

# with sns.plotting_context("paper"):
#     fig, (ax, ax1) = plt.subplots(1, 2, figsize=(13.33/2, 7.5/2.5))
    
#     axs = [ax, ax1]

#     ax.set_xlabel('[Metal]:[EH-Mebip]')
#     ax1.set_xlabel(f'[HCl]:[Complex]')
#     ax.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontdict={'size': 12,})

#     # Remove right and top borders of plots
#     for a in axs:
#         a.spines[['right', 'top', 'left']].set_visible(False)
#         # a.spines[['bottom']].set_linewidth(2)
#         a.tick_params(length=4, axis='x')
#         a.grid(axis='y', linestyle=':', lw=1, color='grey')
#         a.tick_params(axis='y',length=0)
#         a.set_axisbelow(True)

#     # Set colors for graph
#     PalSet = sns.color_palette('deep', len(df.name.unique()))
#     m_colors = PalSet.as_hex()
    
#     # Plot data of interest
#     to_plot=df[['g_h', 'guest_name', 'name', 'value', 'rank']].drop_duplicates()
#     to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]
#     for i, a in enumerate(axs):
#         to_plot_split = to_plot[(to_plot['rank']==i) & (to_plot['name']!='LUB099')] if i ==0 else  \
#                                 to_plot[(to_plot['rank']==i) & (to_plot['name']!='LUB0100')] # Based on titrations performed.
#         sns.lineplot(to_plot_split, x="g_h", y="value", hue='guest_name', ax=a, palette='deep', errorbar='ci') if i == 0 else  \
#                 sns.lineplot(to_plot_split, x="g_h", y="value", hue='name', ax=a, palette='deep', errorbar='ci', legend=False)
#         a.legend(edgecolor='w')

#         #TODO: Subscript all guest names where numbers
        
#         # Scatter plot for hollow spheres - matplotlib
#         for (exp, group), col in zip(to_plot_split.groupby(['name', 'guest_name', 'rank'], as_index=False), m_colors):
#             group_mean = group.groupby('g_h', as_index=False).agg({'value':'mean'})
#             a.scatter(group_mean.g_h, group_mean.value, edgecolors=col, facecolors="none", alpha=0.7, label=exp)

# # Verticle lines indicating orders of complex
# for el in [0.25, 0.5, 1]:
#     ax.axvline(x=el, ls='--', lw=1.5, color='black', alpha=0.4)

# ax1.set_ylabel(None)
# fig.tight_layout()

##### Spectra

In [ ]:
%matplotlib qt
# Plots each titration individually
# Arange data and group plus plot

# TODO: Change pallete, remove legend
to_plot=df[['g_h', 'host_name', 'guest_name', 'name', 'Wavelength nm.', 'Abs.']].drop_duplicates()
to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]

with sns.plotting_context("paper", font_scale=1.75,
                          rc={
                              "lines.linewidth": 1.75, 'lines.markersize': 6.5, 'patch.linewidth': 1.35
                          }):

    for (exp, group) in to_plot.groupby(['name', grouper], as_index=False, sort=False):
        print(exp)
        print(group)
        if 'Eu' not in exp[0]:
            group = group.loc[group['Wavelength nm.'].between(250, 600), :]
            fig, ax = plt.subplots(1) # Set a size once deteremined !
        
            # Background colours
            # BG_WHITE = "#fafaf5"
            # fig.patch.set_facecolor(BG_WHITE)
            # ax.set_facecolor(BG_WHITE)
        
            # Border params
            ax.spines[['right', 'top']].set_visible(False)
            # ax.spines[['bottom']].set_linewidth(2)
            # ax.grid(axis='y', linestyle=':', lw=1, color='grey')
            # ax.set_axisbelow(True)
            
            # axis params
            ax.set_xlabel('Wavelength (nm)')
            ax.set_ylabel('Absorbance (a.u.)')
            ax.xaxis.set_minor_locator(AutoMinorLocator(2))
            ax.yaxis.set_major_locator(MultipleLocator(0.2))
            ax.yaxis.set_minor_locator(AutoMinorLocator(2))
            # ax.tick_params(which="both", bottom=True, top=True, left=True, right=True, direction="in")
            # ax.tick_params(which="both", labelbottom=True, labeltop=False, labelleft=True, labelright=False, direction="in")

            
            # Plot line graphs
            sns.lineplot(data=group, 
                         x="Wavelength nm.", y="Abs.", hue='g_h', palette='flare', ax=ax, legend=False)
            
            # Colourbar 
            norm = plt.Normalize(group.g_h.min(), group.g_h.max())
            sm = plt.cm.ScalarMappable(cmap='flare', norm=norm)
            cbar = fig.colorbar(sm, shrink=0.7, cax=ax.inset_axes((0.85, 0.125, 0.025, 0.75)))
            # cbar.ax.set_title(f'{exp[0]} eq.')
            # cbar.ax.set_title(f'HCl eq.')
            cbar.ax.set_title(f'Fe(III)(OTf)$_{2}$ eq.', pad=10)
            # cbar.ax.set_title(f'Dipea eq.', pad=10)
            
            # ax.set_ylim(ax.get_ylim()[0], 0.84)
            fig.tight_layout()

In [ ]:
# Plot spectra of complexed and decomplexed
with sns.plotting_context("paper"):
    fig, (ax, ax1) = plt.subplots(1, 2, figsize=(13.33/2, 7.5/2.5))
    ax.set_xlabel('Wavelength (nm)')
    
    # Remove right and top borders of plots
    ax.spines[['right', 'top', 'left']].set_visible(False)
    ax.spines[['bottom']].set_linewidth(2)
    ax.tick_params(length=4, axis='x')
    ax.grid(axis='y', linestyle=':', lw=1, color='grey')
    ax.tick_params(axis='y',length=0)
    ax.set_axisbelow(True)


to_plot=df[['g_h', 'guest_name', 'name', 'Wavelength nm.', 'Abs.']].drop_duplicates()
to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]
# to_plot.pivot(index='Wavelength nm.', columns=['name', 'g_h'], values='Abs.')
# Scatter plot for hollow spheres - matplotlib
to_plot = to_plot[to_plot.name == 'LUB045']
for (exp, group) in to_plot.groupby(['name', 'guest_name'], as_index=False):
    fig, ax = plt.subplots(1)
    sns.lineplot(data=group, x="Wavelength nm.", y="Abs.", hue='g_h')
    # group_mean = group.groupby('g_h', as_index=False).agg({'value':'mean'})
    # print(f'{exp} : {col}')
    # ax.scatter(group_mean.g_h, group_mean.value, edgecolors=col, facecolors="none", alpha=0.7, label=exp)
# sns.lineplot(to_plot, x="Wavelength nm.", y="Abs.", hue='g_h', style='name')

#### Reverse Equlibriums

In [ ]:
%matplotlib qt
with sns.plotting_context("paper"):
    # fig, ax = plt.subplots(1, figsize=(13.33/2, 7.5/2.5))
    fig, ax = plt.subplots(1)

    axs = [ax]
    
    ax.set_xlabel(f'[Guest]:[Host]')
    # ax2.set_xlim(-0.1, 0.75)
    # ax2.set_xlim(-0.1, 1.55)
    ax.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontdict={'size': 12,})
    
    # Remove right and top borders of plots
    for a in axs:
        a.spines[['right', 'top', 'left']].set_visible(False)
        a.spines[['bottom']].set_linewidth(2)
        a.tick_params(length=4, axis='x')
        a.grid(axis='y', linestyle=':', lw=1, color='grey')
        a.tick_params(axis='y',length=0)
        a.set_axisbelow(True)

    # Set colors for graph
    PalSet = sns.color_palette('deep', len(df.name.unique()))
    m_colors = ['#4c72b0', '#dd8452', '#4c72b0', '#dd8452'] # Change according to your setup !! Check the print below.
    
    
    # Plot data of interest
    to_plot=df[['g_h', 'guest_name', 'name', 'value', 'rank']].drop_duplicates()
    to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]
    sns.lineplot(to_plot, x="g_h", y="value", hue='guest_name', style='rank', ax=ax, palette=PalSet)
    
    # Scatter plot for hollow spheres - matplotlib
    for (exp, group), col in zip(to_plot.groupby(['name', 'guest_name', 'rank'], as_index=False), m_colors):
        group_mean = group.groupby('g_h', as_index=False).agg({'value':'mean'})
        print(f'{exp} : {col}')
        # ax.scatter(group_mean.g_h, group_mean.value, edgecolors=col, facecolors="none", alpha=0.7, label=exp)
    
    fig.tight_layout()
    # for name in to_plot.name.unique():
    #     print(to_plot[to_plot.name == name])
    #     sns.lineplot(to_plot[to_plot.name == name], x="g_h", y="value", hue='guest_name', style='name', ax=ax)
    

In [ ]:
# Plot data of interest
to_plot=df[['g_h', 'guest_name', 'name', 'Wavelength nm.', 'Abs.']].drop_duplicates()
to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]
# to_plot.pivot(index='Wavelength nm.', columns=['name', 'g_h'], values='Abs.')
# Scatter plot for hollow spheres - matplotlib
to_plot = to_plot[to_plot.name == 'LUB045']
for (exp, group) in to_plot.groupby(['name', 'guest_name'], as_index=False):
    fig, ax = plt.subplots(1)
    sns.lineplot(data=group, x="Wavelength nm.", y="Abs.", hue='g_h')
    # group_mean = group.groupby('g_h', as_index=False).agg({'value':'mean'})
    # print(f'{exp} : {col}')
    # ax.scatter(group_mean.g_h, group_mean.value, edgecolors=col, facecolors="none", alpha=0.7, label=exp)
# sns.lineplot(to_plot, x="Wavelength nm.", y="Abs.", hue='g_h', style='name')

In [ ]:
fig, ax = plt.subplots()
m_colors = ["blue", "tab:orange", "red", "green"]

print(to_plot.groupby(['guest_name', 'rank', 'name'], as_index=False).agg({'g_h':'mean', 'value':'mean'}))

for (exp, group), col in zip(to_plot.groupby(['guest_name', 'rank', 'name'], as_index=False), m_colors):
    print(exp)
    ay = group.groupby('g_h', as_index=False).agg({'value':'mean'})
    # print(group.agg({'g_h':'mean', 'value':'mean'}))
    ax.scatter(ay.g_h, ay.value, edgecolors=col, facecolors="none", alpha=0.7, label=exp)

In [ ]:
# Example of how to plot with new format

to_plot=df[['g_h', 'guest_name', 'name', 'value']].drop_duplicates()
to_plot.loc[:, 'name'] = to_plot.loc[:, 'name'].str.split('_').str[0]
to_plot
for name in to_plot.name.unique():
    print(to_plot[to_plot.name == name])
    sns.lineplot(to_plot[to_plot.name == name], x="g_h", y="value", hue='name', style='guest_name')

In [ ]:
def track_data(df, wl): return df.loc[df['Wavelength nm.']>=250, 'Abs.'].max() if wl=='max' \
                                else df.loc[df['Wavelength nm.'] == wl]['Abs.'].mean()


def construct_init_follow_up_titration(data, guests):
    assert len(guests) <= 2, f'Can only handle two guests, adjust accordingly'
    if len(guests) == 1:
        return data.loc[:, (slice(None), guests[0])]
    
    elif len(guests) == 2:
        # Get only data that has a second titration
        second_tit_name = data.loc[:, (slice(None), guests[-1])].columns.get_level_values('name')
        second_tit_data = data.loc[:, (second_tit_name, guests[0])]
        
        new_vals = second_tit_data.loc[:, (slice(None), guests[0])] \
                       .dropna(how='all', axis=0) \
                       .ffill() \
                       .iloc[-1] \
                       .values
        
        data.loc[0.0, (second_tit_name, guests[-1])] = new_vals
    
    else:
        print(f'Fix your shit')
    
    return data

# Track diff with max peak !!
def tracking_df(df, wls, exps, guests):
    dict_df = {}
    for wl in wls:
        tracking_data = df[np.isin(df.name, exps)] \
                          .groupby(['g_h', 'name', 'titrant']) \
                          .apply(lambda x:  track_data(x, wl) / (track_data(x, 'max'))) \
                          .unstack(['name', 'titrant'])
#                           .apply(lambda x: (track_data(x, 'max') - track_data(x, wl))) \
#                           .apply(lambda x: (track_data(x, 'max')\track_data(x, wl))) \
#                           .unstack(['name', 'titrant'])
        
        print(tracking_data)
        tracking_data = construct_init_follow_up_titration(tracking_data, guests) \
                          .dropna(how='all', axis=0) \
                          .apply(lambda x: x - x.loc[0]) 
                          # .apply(lambda x: (x - x.min())/(x.max() - x.min()))

        # tracking_data = tracking_data.dropna(how='all', axis=0).apply(lambda x: x - x.loc[0]) 
        
        dict_df[str(wl)] = tracking_data
    
    return dict_df

# non_metal_guests = ['hcl', 'teac', 'tfsa', 'tfa']
# non_metal_guests = ['hcl', 'hcl', 'hcl', 'hcl']
# non_metal_guests = ['hcl', 'hcl', 'hcl', 'hcl']
# dict_df = tracking_df(new_data, [340], exps_reps, ['m', non_metal_guests])
dict_df = tracking_df(new_data, [340], exps_reps, ['hcl', 'm'])
# dict_df = tracking_df(new_data, [340], exps_reps, ['m'])
dict_df

In [ ]:
dict_df['340']['LUB005_1'].m.dropna().plot()

### Non-linear regression: Calculation of association constants

In [ ]:
from scipy.optimize import curve_fit
from numpy.polynomial import Polynomial


# Host-guest 1:1 solutions
def func_assoc_11(G, H, Ka, mlr_abs_coeff_hg):
    # Unknowns: Ka, mlr_abs_coeff_hg
    
    hg = 0.5 * (G + H + 1/Ka) - np.sqrt((G - H - 1/Ka)**2 + 4*(G/Ka))
    
    return mlr_abs_coeff_hg * hg


# Host-guest 1:2 solutions
def func_assoc_12(G, H, K1, K2, mlr_abs_coeff_hg, mlr_abs_coeff_hg2):
    # Unknowns: K1, K2, mlr_abs_coeff_hg, mlr_abs_coeff_hg2
    
    a = K1 * K2
    b = K1 * (2 * K2 * H - K2 * G + 1)
    c = K1 * (H - G) + 1

    # roots of cubic polynomial - take minimum real solution
    f_g_roots = np.real(Polynomial(a, b, c).roots())
    f_g = f_g_roots[f_g_roots >= 0].min()

    top_frac = mlr_abs_coeff_hg * H * K1 * f_g + mlr_abs_coeff_hg2 * H * K1 * K2 * f_g**2
    bot_frac = 1 + K1 * f_g + K1 * K2 * f_g**2
    func = top_frac / bot_frac

    return func


# Host-guest 2:1 solutions
def func_assoc_21(G, H, K1, K2, mlr_abs_coeff_hg, mlr_abs_coeff_hg2):
    # Unknowns: K1, K2, mlr_abs_coeff_hg, mlr_abs_coeff_hg2
    
    a = K1 * K2
    b = K1 * (2 * K2 * G - K2 * H + 1)
    c = K1 * (G - H) + 1

    # roots of cubic polynomial - take minimum real solution
    f_h_roots = np.real(Polynomial(a, b, c).roots())
    f_h = f_h_roots[f_h_roots >= 0].min()

    top_frac = mlr_abs_coeff_hg * G * K1 * f_h + mlr_abs_coeff_hg2 * G * K1 * K2 * f_h**2
    bot_frac = 1 + K1 * f_h + K1 * K2 * f_h**2
    func = top_frac / bot_frac

    return func

In [ ]:
df_mock = dict_df['340']['LUB005_1'].m.dropna()
df_mock = df_mock.reset_index()
df_mock.g_h = df_mock.g_h * 0.000021

popt, pcov = curve_fit(func_assoc_12, xdata=df_mock.g_h.values, ydata=df_mock.m.values, p0=[0.000021, 10000, 10000, 0.01, 0.01])

In [ ]:
plt.plot(df_mock.g_h, df_mock.m)
plt.plot(df_mock.g_h, func_assoc_11(df_mock.g_h, *popt), 'r-',label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))
print(popt)

### Visualise peak tracking data

In [ ]:
# %matplotlib qt

# # TODO: This is limited to only two guests, this can be automated to len(guests)
# # TODO: Need to distinguish between guest and metal so can personalise plots, at the moment it is sequential
# # cols_dont_want = [('LUB021_3', 'm'), ('LUB021_6', 'm'), ('LUB038', 'm'), ('LUB069', 'm'), ('LUB069_1', 'm'), ('LUB069_2', 'm')]

# with sns.plotting_context("paper"):
#     fig, ax = plt.subplots(1, figsize=(3.54, 3))
#     fig2, ax2 = plt.subplots(1, figsize=(3.54, 3))
    
#     axs = [ax, ax2]
#     figs = [fig, fig2]

#     ax2.set_xlabel('[Metal]:[EH-Mebip]')
#     ax.set_xlabel(f'[HCl]:[Complex]')
#     # for el in [0.25, 0.5, 1]:
#     #     ax2.axvline(x=el, ls='--', lw=1.5, color='black', alpha=0.4)
#     # ax2.set_xlim(-0.1, 0.75)
#     ax2.set_xlim(-0.1, 1.55)
    
#     # Remove right and top borders of plots
#     for a in axs:
#         a.spines[['right', 'top', 'left']].set_visible(False)
#         a.spines[['bottom']].set_linewidth(2)
#         a.tick_params(length=4, axis='x')
#         a.grid(axis='y', linestyle=':', lw=1, color='grey')
#         a.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontdict={'size': 12,})
#         a.tick_params(axis='y',length=0)
#         a.set_axisbelow(True)
    
#     # leg = ['Zn(OTf)$_{2}$', 'Eu(OTf)$_{3}$', 'Eu(ClO$_{4}$)$_{3}$', 'Cu(OTf)$_{2}$']
#     # leg = ['Zn(OTf)$_{2}$']
#     for k, v in dict_df.items():
#         guest_titrants = v.columns.unique(level='titrant')
#         exps = v.columns.unique(level='name')

#         # Drop columns that you don't want to plot
#         # v = v.drop(columns=cols_dont_want)
        
#         PalSet = sns.color_palette('Set2', len(exps))
#         colors = PalSet.as_hex()
#         dict_non_metal_guest = dict(zip(non_metal_guests, 
#                                         zip(PalSet[:len(non_metal_guests)], colors[:len(non_metal_guests)])))
#         print(dict_non_metal_guest)
        
#         # TODO: Fix for when second level is same but top is different (diff. exp and same titrant)
#         for i, guest in enumerate(guest_titrants):
#             df_guest = v.loc[:, (slice(None), guest)].rename(columns=lambda x: x.split('_')[0])
            
#             for j, col in enumerate(df_guest.columns.unique()):
#                 to_plot = df_guest.dropna() if len(df_guest.columns) == 1 or isinstance(df_guest, pd.Series) \
#                                                 else df_guest[col].dropna()
                
#                 ax = axs[1] if guest=='m' else axs[0]
                
#                 # pal = [PalSet[j]] if guest == 'm' else [dict_non_metal_guest[guest][0]]
#                 pal = [PalSet[j]]
#                 # color = dict_non_metal_guest[guest][1] if guest != 'm' else colors[j]
#                 color = colors[j]
    
                
#                 # to_plot.columns = to_plot.columns.set_levels(leg, level=0)
#                 print(to_plot)
#                 sns.lineplot(data=to_plot, ax=ax, palette=pal, lw=1.75, errorbar='ci', legend=False, label=guest.upper())
#                 # sns.lineplot(data=to_plot, ax=ax, palette=pal, lw=1.75, errorbar='ci', legend=True)
#                 ax.plot(to_plot.mean(axis=1) if len(df_guest.columns) > 1 else \
#                         to_plot, 'o', markerfacecolor='None', markersize=5, markeredgecolor=color)

# for i, a in enumerate(axs):
#     a.legend(edgecolor='w')
#     figs[i].tight_layout()

# # axs[1].legend(leg, edgecolor='w')

# for el in [0.25, 0.5, 1]:
#     ax2.axvline(x=el, ls='--', lw=1.5, color='black', alpha=0.4)
    
# # axs[0].set_xlim(0, 2)

In [ ]:
# TODO: This is limited to only two guests, this can be automated to len(guests)
# TODO: Need to distinguish between guest and metal so can personalise plots, at the moment it is sequential
# cols_dont_want = [('LUB021_3', 'm'), ('LUB021_6', 'm'), ('LUB038', 'm'), ('LUB069', 'm'), ('LUB069_1', 'm'), ('LUB069_2', 'm')]
%matplotlib qt
with sns.plotting_context("paper"):
    # fig, (ax2, ax) = plt.subplots(1, 2, figsize=(6.5, 3))
    # fig, (ax2, ax) = plt.subplots(1, 2, figsize=(13.33/2, 7.5/2.15))
    fig, (ax2, ax) = plt.subplots(1, 2, figsize=(13.33/2, 7.5/2.5))
    
    axs = [ax, ax2]

    # ax2.set_xlabel('[Eu$^{3+}$]:[EH-Mebip]')
    ax2.set_xlabel('[Metal]:[EH-Mebip]')
    ax.set_xlabel(f'[HCl]:[Complex]')
    # for el in [0.25, 0.5, 1]:
    #     ax2.axvline(x=el, ls='--', lw=1.5, color='black', alpha=0.4)
    ax2.set_xlim(-0.1, 0.75)
    ax2.set_xlim(-0.1, 1.55)
    ax2.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontdict={'size': 12,})
    
    # Remove right and top borders of plots
    for a in axs:
        a.spines[['right', 'top', 'left']].set_visible(False)
        a.spines[['bottom']].set_linewidth(2)
        a.tick_params(length=4, axis='x')
        a.grid(axis='y', linestyle=':', lw=1, color='grey')
        # a.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontdict={'size': 12,})
        a.tick_params(axis='y',length=0)
        a.set_axisbelow(True)
    
    leg = ['Zn(OTf)$_{2}$', 'Eu(ClO$_{4}$)$_{3}$', 'Cu(OTf)$_{2}$', 'Eu(OTf)$_{3}$']
    cols_dont_want = [('LUB099', 'm'), ('LUB099_1', 'm')]
    # leg = ['Zn(OTf)$_{2}$']
    # leg = ['0.5 Eu(OTf)$_{3}$', '0.25 Eu(OTf)$_{3}$', 'Excess Eu(OTf)$_{3}$']
    for k, v in dict_df.items():
        guest_titrants = v.columns.unique(level='guest_name')
        exps = v.columns.unique(level='name')

        # Drop columns that you don't want to plot
        v = v.drop(columns=cols_dont_want)
        
        PalSet = sns.color_palette('deep', len(exps))
        colors = PalSet.as_hex()
        dict_non_metal_guest = dict(zip(non_metal_guests, 
                                        zip(PalSet[:len(non_metal_guests)], colors[:len(non_metal_guests)])))
        
        # TODO: Fix for when second level is same but top is different (diff. exp and same titrant)
        for i, guest in enumerate(guest_titrants):
            df_guest = v.loc[:, (slice(None), guest)].rename(columns=lambda x: x.split('_')[0])
            
            for j, col in enumerate(df_guest.columns.unique()):
                to_plot = df_guest.dropna() if len(df_guest.columns) == 1 or isinstance(df_guest, pd.Series) \
                                                else df_guest[col].dropna()
                
                ax = axs[1] if guest=='m' else axs[0]
                
                # pal = [PalSet[j]] if guest == 'm' else [dict_non_metal_guest[guest][0]]
                pal = [PalSet[j]]
                # color = dict_non_metal_guest[guest][1] if guest != 'm' else colors[j]
                color = colors[j]
    
                
                print(to_plot)
                to_plot.columns = to_plot.columns.set_levels(leg, level=0)
                print(to_plot)
                to_plot = to_plot[to_plot.index <= 1.5] if guest=='m' else to_plot
                # sns.lineplot(data=to_plot, ax=ax, palette=pal, lw=1.75, errorbar='ci', legend=False, label=guest.upper())
                sns.lineplot(data=to_plot, ax=ax, palette=pal, errorbar='ci', legend=True if guest=='m' else False)
                ax.plot(to_plot.mean(axis=1) if len(df_guest.columns) > 1 else \
                        to_plot, 'o', markerfacecolor='None', markersize=4.5, markeredgecolor=color)

for i, a in enumerate(axs):
    print(i)
    if i == 1:
        a.legend(['1']*4, edgecolor='w')
    a.legend(edgecolor='w')
    # figs[i].tight_layout()

# axs[1].legend(leg, edgecolor='w')

for el in [0.25, 0.5, 1]:
    ax2.axvline(x=el, ls='--', lw=1.5, color='black', alpha=0.4)

fig.tight_layout()
# axs[0].set_xlim(0, 2)

In [ ]:
# TODO: This is limited to only two guests, this can be automated to len(guests)
# TODO: Need to distinguish between guest and metal so can personalise plots, at the moment it is sequential
# cols_dont_want = [('LUB021_3', 'm'), ('LUB021_6', 'm'), ('LUB038', 'm'), ('LUB069', 'm'), ('LUB069_1', 'm'), ('LUB069_2', 'm')]
%matplotlib qt
with sns.plotting_context("paper"):
    # fig, (ax2, ax) = plt.subplots(1, 2, figsize=(6.5, 3))
    # fig, (ax2, ax) = plt.subplots(1, 2, figsize=(13.33/2, 7.5/2.15))
    fig, (ax2, ax) = plt.subplots(1, 2, figsize=(13.33/2, 7.5/2.5))
    
    axs = [ax, ax2]

    # ax2.set_xlabel('[Eu$^{3+}$]:[EH-Mebip]')
    ax2.set_xlabel('[Metal]:[EH-Mebip]')
    ax.set_xlabel(f'[HCl]:[Complex]')
    # for el in [0.25, 0.5, 1]:
    #     ax2.axvline(x=el, ls='--', lw=1.5, color='black', alpha=0.4)
    ax2.set_xlim(-0.1, 0.75)
    ax2.set_xlim(-0.1, 1.55)
    ax2.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontdict={'size': 12,})
    
    # Remove right and top borders of plots
    for a in axs:
        a.spines[['right', 'top', 'left']].set_visible(False)
        a.spines[['bottom']].set_linewidth(2)
        a.tick_params(length=4, axis='x')
        a.grid(axis='y', linestyle=':', lw=1, color='grey')
        # a.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontdict={'size': 12,})
        a.tick_params(axis='y',length=0)
        a.set_axisbelow(True)
    
    leg = ['Zn(OTf)$_{2}$', 'Eu(ClO$_{4}$)$_{3}$', 'Cu(OTf)$_{2}$', 'Eu(OTf)$_{3}$']
    cols_dont_want = [('LUB099', 'm'), ('LUB099_1', 'm')]
    # leg = ['Zn(OTf)$_{2}$']
    # leg = ['0.5 Eu(OTf)$_{3}$', '0.25 Eu(OTf)$_{3}$', 'Excess Eu(OTf)$_{3}$']
    for k, v in dict_df.items():
        guest_titrants = v.columns.unique(level='titrant')
        exps = v.columns.unique(level='name')

        # Drop columns that you don't want to plot
        v = v.drop(columns=cols_dont_want)
        
        PalSet = sns.color_palette('deep', len(exps))
        colors = PalSet.as_hex()
        dict_non_metal_guest = dict(zip(non_metal_guests, 
                                        zip(PalSet[:len(non_metal_guests)], colors[:len(non_metal_guests)])))
        
        # TODO: Fix for when second level is same but top is different (diff. exp and same titrant)
        for i, guest in enumerate(guest_titrants):
            df_guest = v.loc[:, (slice(None), guest)].rename(columns=lambda x: x.split('_')[0])
            
            for j, col in enumerate(df_guest.columns.unique()):
                to_plot = df_guest.dropna() if len(df_guest.columns) == 1 or isinstance(df_guest, pd.Series) \
                                                else df_guest[col].dropna()
                
                ax = axs[1] if guest=='m' else axs[0]
                
                # pal = [PalSet[j]] if guest == 'm' else [dict_non_metal_guest[guest][0]]
                pal = [PalSet[j]]
                # color = dict_non_metal_guest[guest][1] if guest != 'm' else colors[j]
                color = colors[j]
    
                
                print(to_plot)
                to_plot.columns = to_plot.columns.set_levels(leg, level=0)
                print(to_plot)
                to_plot = to_plot[to_plot.index <= 1.5] if guest=='m' else to_plot
                # sns.lineplot(data=to_plot, ax=ax, palette=pal, lw=1.75, errorbar='ci', legend=False, label=guest.upper())
                sns.lineplot(data=to_plot, ax=ax, palette=pal, errorbar='ci', legend=True if guest=='m' else False)
                ax.plot(to_plot.mean(axis=1) if len(df_guest.columns) > 1 else \
                        to_plot, 'o', markerfacecolor='None', markersize=4.5, markeredgecolor=color)

for i, a in enumerate(axs):
    print(i)
    if i == 1:
        a.legend(['1']*4, edgecolor='w')
    a.legend(edgecolor='w')
    # figs[i].tight_layout()

# axs[1].legend(leg, edgecolor='w')

for el in [0.25, 0.5, 1]:
    ax2.axvline(x=el, ls='--', lw=1.5, color='black', alpha=0.4)

fig.tight_layout()
# axs[0].set_xlim(0, 2)

In [ ]:
%matplotlib qt

# TODO: This is limited to only two guests, this can be automated to len(guests)
# TODO: Need to distinguish between guest and metal so can personalise plots, at the moment it is sequential
with sns.plotting_context("paper"):
    fig, ((ax1, ax2), (ax4, ax3)) = plt.subplots(2, 2, figsize=(10.67, 6.5))
    # fig, ((ax1, ax2), (ax4, ax3)) = plt.subplots(2, 2, figsize=(13.33, 7.5))
    # fig2, ax2 = plt.subplots(1, figsize=(3.54, 3))
    
    axs = [ax3, ax4]
    figs = [fig]

    ax4.set_xlabel('[Zn$^{2+}$]:[EH-Mebip]')
    ax3.set_xlabel('[Acid]:[Zn(EH-Mebip)$_{2}$]')
    # for el in [0.25, 0.5, 1]:
    #     ax4.axvline(x=el, ls='--', lw=1.5, color='black', alpha=0.4)
    ax4.set_xlim(-0.1, 0.75)
    # ax4.set_xlim(-0.1, 2)
    ax4.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontdict={'size': 12,})
    
    # Remove right and top borders of plots
    for a in axs:
        a.spines[['right', 'top', 'left']].set_visible(False)
        a.spines[['bottom']].set_linewidth(2)
        a.tick_params(length=4, axis='x')
        a.grid(axis='y', linestyle=':', lw=1, color='grey')
        # fig.supylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$')
        # a.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontdict={'size': 12,})
        a.tick_params(axis='y',length=0)
        a.set_axisbelow(True)
    
    # leg = ['Zn(OTf)$_{2}$', 'Eu(OTf)$_{3}$', 'Eu(ClO$_{4}$)$_{3}$', 'Cu(OTf)$_{2}$']
    leg = ['Zn(OTf)$_{2}$']
    cols_dont_want = [('LUB021_3', 'm'), ('LUB021_6', 'm'), ('LUB038', 'm'), ('LUB038_6', 'm'), ('LUB069', 'm'), ('LUB069_1', 'm'), ('LUB069_2', 'm')]
    
    for k, v in dict_df.items():
        guest_titrants = v.columns.unique(level='titrant')
        exps = v.columns.unique(level='name')

         # Drop columns that you don't want to plot
        v = v.drop(columns=cols_dont_want)
        
        PalSet = sns.color_palette('deep', len(exps))
        colors = PalSet.as_hex()
        dict_non_metal_guest = dict(zip(non_metal_guests, 
                                        zip(PalSet[:len(non_metal_guests)], colors[:len(non_metal_guests)])))
        
        # TODO: Fix for when second level is same but top is different (diff. exp and same titrant)
        for i, guest in enumerate(guest_titrants):
            df_guest = v.loc[:, (slice(None), guest)].rename(columns=lambda x: x.split('_')[0])
            
            for j, col in enumerate(df_guest.columns.unique()):
                to_plot = df_guest.dropna() if len(df_guest.columns) == 1 or isinstance(df_guest, pd.Series) \
                                                else df_guest[col].dropna()
                
                ax = axs[1] if guest=='m' else axs[0]
                
                pal = [PalSet[j]] if guest == 'm' else [dict_non_metal_guest[guest][0]]
                color = dict_non_metal_guest[guest][1] if guest != 'm' else colors[j]
    
                
                # to_plot.columns = to_plot.columns.set_levels(leg, level=0)
                sns.lineplot(data=to_plot, ax=ax, palette=pal, lw=1, errorbar='ci', legend=False, label=guest.upper() if 'hcl' not in guest else 'HCl')
                # sns.lineplot(data=to_plot, ax=ax, palette=pal, lw=1.75, errorbar='ci', legend=True)
                ax.plot(to_plot.mean(axis=1) if len(df_guest.columns) > 1 else \
                        to_plot, 'o', markerfacecolor='None', markersize=4.5, markeredgecolor=color)

for i, a in enumerate(axs):
    a.legend(edgecolor='w')

ax4.legend(leg, edgecolor='w')

for el in [0.25, 0.5, 1]:
    ax4.axvline(x=el, ls='--', lw=1.5, color='black', alpha=0.4)
    # figs[i].tight_layout()
# fig.tight_layout()
# axs[0].legend(leg)
# axs[0].set_xlim(0, 2)

In [ ]:
%matplotlib qt
# d = new_data.drop_duplicates(['Wavelength nm.', 'g_h', 'name', 'titrant'])
# plt.plot(d[(d.name.str.contains('LUB005')) & (d.titrant == 'm')].pivot(index='Wavelength nm.', columns='g_h', values='Abs.'))
# presentation = {xlabel: {fontdict={'size': 12,}, labelpad=7.5}, ylabel=fontdict={'size': 12,}, labelpad=15, lw:2, markersize=5}
n = 15
c=0.4
colors = (1. - c) * plt.get_cmap("Blues_r")(np.linspace(0., 1., n)) + c * np.ones((n, 4))
exp_n = exp_names[0] # Plot one
# exp_n = 'LUB005_1'
axs_2 = [ax1, ax2]
for name in [exp_n]:
    data_exp = new_data[new_data.name.str.contains(name)]
    data_exp.loc[:, 'g_h'].round(2) == data_exp.g_h.round(2)

    # plot data for each guest in exp
    for guest in data_exp.titrant.unique():

        ax = ax1 if guest=='m' else ax2
    
        with sns.plotting_context("paper"):
            # fig, ax = plt.subplots(figsize=(3.54, 3.54))
            
            # axis of uvvis
            ax.set_xlabel('Wavelength (nm)')
            
            # Remove right and top borders of plots
            ax.spines[['right', 'top', 'left']].set_visible(False)
            ax.spines[['bottom']].set_linewidth(2)
            ax.tick_params(length=4, axis='x')
            ax.grid(axis='y', linestyle=':', lw=1, color='grey')
            ax.tick_params(axis='y',length=0)
            ax.set_axisbelow(True)
        
            # # Remove right and top borders of plots
            # ax.spines[['right', 'top']].set_visible(False)
            
            # pivot data accordingly
            to_plot = data_exp[data_exp.titrant == guest].pivot(index='Wavelength nm.', columns=['name', 'g_h'], values='Abs.')
        
            sns.lineplot(to_plot.droplevel('name', axis=1), dashes=False, legend=False, palette='Spectral', ax=ax)
            # fig.tight_layout()
ax1.set_ylabel('Absorbance (A)')
ax2.set_ylim(ax1.get_ylim())
fig.tight_layout()
            # sns.lineplot(data=lb5, x='Wavelength nm.', y='')

## NMR Titration - Temporary

In [ ]:
import matplotlib

f = matplotlib.font_manager.FontManager()
f.findfont("Comic Neue")
matplotlib.font_manager._load_fontmanager(try_read_cache=False)

In [ ]:
exp_names = ['LUB056']

hera.initialize_titrations(exp_names)

dict_titrants_exps = hera.dict_titrants_exps.copy()
dict_titrations_exps = deepcopy(hera.dict_titrations_exps)
dict_titrants_exps

In [ ]:
for k, v in dict_titrations_exps.items():
    print(f'\n\n{k}:\n')
    for t in v[0]:
        print(t.df_params)

In [ ]:
nmr_track_data ={
    "H3": [7.72, 7.77, 7.84, 7.88, 7.94, 7.95, 7.99, 8.01, 8.02, 8.00, 8.01, 8.01, 8.00, 8.00, 7.99, 7.99],
    "H5": [7.87, 7.90, 7.95, 7.98, 8.00, 8.01, 8.01, 8.00, 7.98, 7.97, 7.96, 7.95, 7.94, 7.93, 7.92, 7.92],
    "H5_2": [7.85, 7.86, 7.90, 7.99, 8.00, 7.99, 7.96, 7.95, 7.93, 7.92, 7.93, 7.91, 7.91, 7.91, 7.90, 7.90],
    "singlet_up": [4.207, 4.199, 4.208, 4.218, 4.223, 4.227, 4.234, 4.238, 4.244, 4.247, 4.248, 4.250, 4.251, 4.250, 4.249, 4.250],
    "acid_up": [2.13, 3.02, 3.82, 4.36, 4.75, 5.01, 5.38, 5.72, 6.01, 6.26, 6.40, 6.62, 6.80, 5.62, 5.56, 5.71],
    "H3_2": [7.47, 7.51, 7.56, 7.60, 7.64, 7.69, 7.72, 7.75, 7.77, 7.78, 7.79, 7.79, 7.80, 7.82, 7.82, 7.82],
    "multi_up": [7.29, 7.35, 7.38, 7.42, 7.47, 7.50, 7.55, 7.58, 7.59, 7.61, 7.61, 7.62, 7.63, 7.63, 7.63, 7.64]
}

nmr_track_data_lub056 ={
    "H5": [7.94, 7.95, 8.00, 8.08, 8.09, 8.08, 8.06, 8.05, 8.03, 8.02, 8.02, 8.01, 8.01, 8.00, 8.0, 8.0, 
           7.99, 7.99, 7.98, 7.97, 7.96, 7.96, 7.93, 7.92, 7.92, 7.92, 7.91]
}

nmr_track = pd.DataFrame(data=nmr_track_data_lub056)
print(nmr_track)
# nmr_track = nmr_track.loc[:, [col for col in nmr_track.columns if 'up' in col or 'down' in col]]
nmr_track = (nmr_track - nmr_track.loc[0, :])/nmr_track.loc[0, :]

nmr_track = pd.concat([t.df_params, pd.DataFrame(data=nmr_track)], axis=1)

In [ ]:
%matplotlib qt
import matplotlib.ticker as ticker

p = sns.color_palette('deep')

colors = {
    "H3": p[3],
    "H5": p[2],
    "H5_2": p[2],
    "singlet_up": p[1],
    "acid_up": p[6],
    "H3_2": p[3],
    "multi_up": p[7]
}

to_p = nmr_track.loc[:, [col for col in nmr_track.columns if 'up' in col or 'down' in col or 'g_h' in col]].set_index('g_h')
to_p = nmr_track.loc[:, [col for col in nmr_track.columns if 'H' in col or 'g_h' in col]].set_index('g_h')
to_p_norm = (to_p - to_p.min())/(to_p.max() - to_p.min())

print(to_p_norm)

peaks_to_plot = ['H3', 'H5']

In [ ]:
# Plot data
with sns.plotting_context("talk"):
    # fig, ax = plt.subplots(1, figsize=(3.54, 3))
    fig, ax = plt.subplots(1, figsize=(8.54, 7))
    
    ax.spines[['right', 'top', 'left']].set_visible(False)
    ax.spines[['bottom']].set_linewidth(2)
    xlim = (-0.3, 8)
    ax.set_xlim(xlim)
    
    ax.tick_params(length=4, axis='x')
    ax.grid(axis='y', linestyle=':', lw=1, color='grey')
    ax.set_ylabel('\u0394 \u03B4(ppm)', labelpad=15)
    ax.set_xlabel('[TFA]:[EH-Mebip]', labelpad=15)
    ax.tick_params(axis='y',length=0)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.set_axisbelow(True)

    
    for col in to_p_norm.columns:
        if col in peaks_to_plot:
            # sns.lineplot(to_p_norm.loc[:, col], color=colors[col]) # lw=1.75 paper
            ax.plot(to_p_norm.loc[:, col], 'o', color=colors[col], label=col, markerfacecolor='None', 
                    markeredgecolor=colors[col], markeredgewidth=2.5, markersize=12)
            ax.legend()
    
    legend = ax.legend(edgecolor='w', loc='lower right')
    # legend.texts[0].set_text("wt % MBTT")
fig.tight_layout()
fig.show()

### Photoacid experiment - MBTT

In [ ]:
exp_names = ['LUB111', 'LUB198']
# exp_names = ['LUB224']
# exp_names = ['LUB220', 'LUB228', 'LUB198']
# exp_names = ['LUB228']
# exp_names = ['LUB200', 'LUB201']

# Need to couple exp number with complex formed
hera.initialize_titrations(exp_names)

dict_titrants_exps = hera.dict_titrants_exps.copy()
dict_titrations_exps = deepcopy(hera.dict_titrations_exps)

In [ ]:
for k, v in dict_titrations_exps.items():
    print(f'\n\n{k}:\n')
    for t in v[0]:
        print(t.df_params)

In [ ]:
dict_titrations_exps['LUB224'][0][0].df_params.guest_conc.iloc[1] * 1e6

In [ ]:
0.5/dict_titrations_exps['LUB228'][0][1].df_params.iloc[0].guest_conc

In [ ]:
#TODO: Check parser get the first line of text file.

exps_reps = parse.get_exp_reps(exp_names)
data = pa.get_data(pa.path_constructor('uv', *exps_reps), 'uv')
data

In [ ]:
# clean up data to only take timestamp not prior complexing and rename
data = data[~data['titrant'].str.contains('i+$', regex=True)].rename(columns={'titrant': 'Time (min)'})

In [ ]:
data

In [ ]:
# Tracking helper function
def track_data(df, wl): return df.loc[df['Wavelength nm.']>=250, 'Abs.'].max() if wl=='max' \
                                else df.loc[df['Wavelength nm.'] == wl]['Abs.'].mean()

wl= 335
# Monitor change in Abs of pi-pi*/MLCT for different experiments and thier times
mbtt_ts = data.apply(pd.to_numeric, errors='ignore') \
              .groupby(['name', 'Time (min)']) \
              .apply(lambda x:  track_data(x, wl) / (track_data(x, 'max')))

# mbtt = 372
if False:
    wl = 372
    mbtt_ts = data.apply(pd.to_numeric, errors='ignore') \
              .groupby(['name', 'Time (min)']) \
              .apply(lambda x:  track_data(x, wl))

# Get difference in abs from starting abs for each experiment
mbtt_ts = mbtt_ts.groupby(['name']) \
                 .apply(lambda x: x - x.iloc[0]) \
                 .droplevel(0) \
                 .reset_index() \
                 .rename(columns={0:'value'})


# Specific for mbtt eq. comparison
# mbtt_ts = mbtt_ts.groupby(['name']) \
#                  .apply(lambda x: x - x.iloc[0]) \
#                  .droplevel(0) \
#                  .reindex(level=0, index=['LUB220', 'LUB228_0', 'LUB228_1', 'LUB198_1']) \
#                  .reset_index() \
#                  .rename(columns={0:'value'})

print(mbtt_ts)
mbtt_ts

In [ ]:
mbtt_ts.name = mbtt_ts.name.apply(lambda x: x.split('_')[0])

In [ ]:
PalSet = sns.color_palette('deep', len(mbtt_ts.name.unique()))
m_colors = PalSet.as_hex()
print(m_colors)
# new_m_colors = [m_colors[0], m_colors[2], m_colors[1]]
# new_m_colors

In [ ]:
%matplotlib qt

legend_labels = ['1.2', '190']
# legend_labels = ['190']
# legend_labels = ['0.17', '1.00', '5.00']
# legend_labels = ['1.00']

# Plot data
with sns.plotting_context("paper", font_scale=1.75, 
                          rc={
                              "lines.linewidth": 2, 'lines.markersize': 8, 'patch.linewidth': 1.825
                          }):
    
    # Set colors for graph
    PalSet = sns.color_palette('deep', len(mbtt_ts.name.unique()))
    m_colors = PalSet.as_hex()

    # Gradient colors for graph
    if True:
        PalSet = sns.color_palette('mako_r', len(mbtt_ts.name.unique()))
        m_colors = PalSet.as_hex()
        m_colors = ['#8fa7ce', '#4c72b0', '#30476e'] # Custom blue palette
    # new_m_colors = [m_colors[2], m_colors[0], m_colors[1]]
    # figsize=(13.33, 6) 
    
    fig, ax = plt.subplots(1)


    ax.spines[['right', 'top']].set_visible(False)
    ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))
    # ax.xaxis.set_major_locator(MultipleLocator(20))

    # Time series plot
    # ax1.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontdict={'size': 12,})
    ax.set_ylabel(r'$\Delta\left(\frac{A_{MLCT}}{A_{\pi\rightarrow\pi^{*}}}\right)$', fontsize=20)
    # ax.set_ylabel(r'${\Delta}A_{\pi\rightarrow\pi^{*}}$')
    # ax.set_ylabel(r'${\Delta}A_{372nm}$')
    ax.set_xlabel('Time (min)')

    # pLot line graphs
    # lnplt = sns.lineplot(mbtt_ts, x='Time (min)', y='value', hue='name', palette=m_colors, legend=True, hue_order=['LUB220', 'LUB228', 'LUB198'])
    lnplt = sns.lineplot(mbtt_ts, x='Time (min)', y='value', hue='name', palette=m_colors, legend=True)
    
    # Legend - manual if legend required :(
    new_title = 'Power density\n(mW/cm$^2$)'
    # new_title = 'MBTT eq.'
    # new_title = 'MBTT conc.'
    bbox_to_anchor=(0.55, 0.45, 0.5, 0.5)
    ax.legend(edgecolor='w', framealpha=0, loc='best', alignment='left')
    lnplt.legend_.set_title(new_title)
    # Replace labels
    for text, label in zip(lnplt.legend_.texts, legend_labels):
        text.set_text(label)

    for (name, df), col in zip(mbtt_ts.groupby(['name'], as_index=False, sort=False), m_colors):
        print(df)
        df_mean = df.groupby('Time (min)', as_index=False).agg({'value':'mean'})
        ax.scatter(df_mean['Time (min)'], df_mean['value'], edgecolors=col, facecolors="none", alpha=0.7, label=name)
    
    # legend = ax.legend(edgecolor='w', title='Time (min)')
fig.tight_layout()

##### Spectra

In [ ]:
with sns.plotting_context("paper", font_scale=1.75, 
                          rc={
                              "lines.linewidth": 2, 'lines.markersize': 8, 'patch.linewidth': 1.825
                          }):

    data.name = data.name.apply(lambda x: x.split('_')[0])
    for (exp, group) in data.groupby(['name'], as_index=False, sort=False):

        # Cut wavelength range
        group = group.loc[group['Wavelength nm.'].between(250, 500), :]
        
        fig, ax = plt.subplots(1)

        ax.spines[['right', 'top']].set_visible(False)
        ax.xaxis.set_minor_locator(AutoMinorLocator(2))
        ax.yaxis.set_major_locator(MultipleLocator(0.2))
        ax.yaxis.set_minor_locator(AutoMinorLocator(2))
    
        # UV Abosrbance stack plot
        ax.set_ylabel('Abosrbance (a.u.)')
        ax.set_xlabel('Wavelength (nm)')
        sns.lineplot(data=group, x="Wavelength nm.", y="Abs.", hue='Time (min)', palette='flare', ax=ax, legend=True)
    
        ax.legend(ncol=1, title='Time (min)', frameon=0)
    
        print(ax.get_ylim())
        # ax.set_ylim(ax.get_ylim()[0], 0.82)
    
        # Colourbar
        # time_scale = [*map(int, group['Time (min)'])]
        # norm = plt.Normalize(min(time_scale), max(time_scale))
        # sm = plt.cm.ScalarMappable(cmap='flare', norm=norm)
        # cbar = fig.colorbar(sm, shrink=0.7, cax=ax.inset_axes((0.85, 0.125, 0.025, 0.75)))
        # cbar.ax.set_title('Time (min)', pad=15)
print(ax.get_ylim())

fig.tight_layout()